### Import the required libraries.

In [1]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from retrieve_data import retrieve_data
from clean_data import clean_data
from prepare_data_for_model import get_data_for_model
from create_the_model import create_model, train_model
from helper import sort_by_game

from tensorflow.keras import models

In [2]:
# retrieve the latest data
data = retrieve_data()

# save latest dataset
data.to_csv('../Data/data.csv', index=False)

<ipython-input-2-a1eee19c3c8a>:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  data = retrieve_data()


In [18]:
# read the data
data = pd.read_csv("../Data/data.csv")

# clean the data
cleaned_data = clean_data(data)

#cleaned_data = cleaned_data.dropna(axis=0, how='any')

# save the cleaned dataset
cleaned_data.to_csv('../Data/cleaned_dataset.csv', index=False)

In [ ]:
for col in cleaned_data.columns:
    print("{:>30} : {}".format(col, cleaned_data.dtypes[col]))

In [ ]:
cleaned_data.shape

### Get training and testing sets.

In [19]:
# read the cleaned data
cleaned_data = pd.read_csv('../Data/cleaned_dataset.csv')

# obtain relevant training and testing datasets
training_data, testing_data = get_data_for_model(cleaned_data)
x_tr, y_tr, y_oh_tr, info_tr, match_ids_tr, player_names_tr = training_data
x_ts, y_ts, y_oh_ts, info_ts, match_ids_ts, player_names_ts = testing_data

# save the training datasets
x_tr.to_csv('../Data/x_tr.csv', index=False)
y_tr.to_csv('../Data/y_tr.csv', index=False)
y_oh_tr.to_csv('../Data/y_oh_tr.csv', index=False)
info_tr.to_csv('../Data/info_tr.csv', index=False)
#match_ids_tr.to_csv('../Data/match_ids_tr.csv', index=False)
#player_names_tr.to_csv('../Data/player_names_tr.csv', index=False)

# save the testing datasets
x_ts.to_csv('../Data/x_ts.csv', index=False)
y_ts.to_csv('../Data/y_ts.csv', index=False)
y_oh_ts.to_csv('../Data/y_oh_ts.csv', index=False)
info_ts.to_csv('../Data/info_ts.csv', index=False)
#match_ids_ts.to_csv('../Data/match_ids_ts.csv', index=False)
#player_names_ts.to_csv('../Data/player_names_ts.csv', index=False)

In [22]:
x_ts.shape

(9108, 54)

In [24]:
np.any(np.isnan(x_tr))

True

In [ ]:

pd.set_option('display.max_columns', None)
x_ts.head(1)

In [ ]:
x_ts.shape

In [16]:
x_ts_2  =x_ts.dropna(axis=0, how='any')

In [17]:
x_ts_2.shape

(9006, 54)

### Train the model.

In [26]:
# read the training data
x_tr = pd.read_csv('../Data/x_tr.csv')
y_oh_tr = pd.read_csv('../Data/y_oh_tr.csv')

# read the testing data
x_ts = pd.read_csv('../Data/x_ts.csv')
y_oh_ts = pd.read_csv('../Data/y_oh_ts.csv')

In [30]:
# create the model
model = create_model()

# train the model
history = train_model(model, x_tr, y_oh_tr, 100, 500, 0.2)

TypeError: fit() got an unexpected keyword argument 'class_weights'

In [ ]:
# save the model
model.save('saved_models/model')

### View model results.

In [ ]:
# load the model
model = models.load_model('saved_models/model')

In [ ]:
# analyse the model
plt.figure(figsize=(12,6))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.savefig('Training_Model.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# check accuracy of model on training & testing data
print(model.evaluate(x_tr, y_oh_tr)[1])
print(model.evaluate(x_ts, y_oh_ts)[1])

In [ ]:
# get probabilities on the training set
probs_tr = model.predict(x_tr)

# sort probabilities by game
#probs_tr_game = sort_by_game(probs_tr, info_tr)

# get expected votes from predicted probabilities & scale expected votes
#exp_votes_tr_game_raw = get_expected_votes(probs_tr_game)
#exp_votes_tr_game = scale_expected_votes(exp_votes_tr_game_raw)

# compute the game predictions based on expected votes
#pred_votes_tr_game = compute_predictions(exp_votes_tr_game)

# see confusion matrix based on predicted votes 
#see_confusion_matrix(pred_votes_tr_game, y_tr_game)

In [ ]:
y_oh_tr

In [ ]:
probs_tr

In [ ]:
y_tr

In [ ]:
# get probabilities on the testing set
probs_ts = model.predict(x_ts)

# sort probabilities by game
probs_ts_game = sort_by_game(probs_ts, info_ts, game_ids_ts)

# get expected votes from predicted probabilities & scale expected votes
exp_votes_ts_game_raw = get_expected_votes(probs_ts_game)
exp_votes_ts_game = scale_expected_votes(exp_votes_ts_game_raw)

# compute the game predictions based on expected votes
pred_votes_ts_game = compute_predictions(exp_votes_ts_game)

# see confusion matrix based on predicted votes 
see_confusion_matrix(pred_votes_ts_game, y_ts_game)

In [ ]:
# show a prediction from the testing dataset
game_num = 25
show_game_prediction(probs_ts_game,
                     exp_votes_ts_game_raw,
                     exp_votes_ts_game, 
                     pred_votes_ts_game, 
                     y_ts_game, 
                     info_ts_game, 
                     game_num)

In [ ]:
# sort expected votes by player
exp_votes_ts_player = sort_votes_by_player(exp_votes_ts_game, info_ts_game, player_names_ts)

# sort predictions by player
pred_votes_ts_player = sort_votes_by_player(pred_votes_ts_game, info_ts_game, player_names_ts)

# compute each player's total expected votes for the 2020 season
exp_votes_ts_player_total = compute_total_votes(exp_votes_ts_player)

# compute each player's total predicted votes for the 2020 season
pred_votes_ts_player_total = compute_total_votes(pred_votes_ts_player)

In [ ]:
# see best performing players based on expected votes
show_most_votes(exp_votes_ts_player_total, player_names_ts, 10)

In [ ]:
# see best performing players based on predicted votes
show_most_votes(pred_votes_ts_player_total, player_names_ts, 10)